In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
import os


from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import wandb_tracing_enabled

In [12]:
chat = ChatOpenAI(temperature=0)
prompt_template = "Tell me a {adjective} joke"
llm_chain = LLMChain(llm=chat, prompt=PromptTemplate.from_template(prompt_template))

llm_chain(inputs={"adjective": "childish"})

{'adjective': 'childish',
 'text': "Why don't scientists trust atoms?\n\nBecause they make up everything!"}

In [6]:
# Agent run with tracing. Ensure that OPENAI_API_KEY is set appropriately to run this example.

llm = OpenAI(temperature=0)
tools = load_tools(["llm-math"], llm=llm)

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run("What is 2 raised to .123243 power?")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Streaming LangChain activity to W&B at https://wandb.ai/datamastery/langchain-tracing/runs/p86fu8l8
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.




> Entering new AgentExecutor chain...
 I need to use a calculator to solve this.
Action: Calculator
Action Input: 2^.123243
Observation: Answer: 1.0891804557407723
Thought: I now know the final answer.
Final Answer: 1.0891804557407723

> Finished chain.


'1.0891804557407723'

In [13]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import YoutubeLoader
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:

llm = ChatOpenAI(temperature=0)

map_template = """
The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

reduce_template = """
The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max=4000,
)
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="docs",
    return_intermediate_steps=False,
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
loader = YoutubeLoader.from_youtube_url("https://youtu.be/fSAw-y9OwD4", add_video_info=True)
docs = loader.load()
split_docs = text_splitter.split_documents(docs)
result = map_reduce_chain.run(split_docs)

print("Processed Result:", result)


Processed Result: The main themes in the given list of documents include the relevance and benefits of Lang chain in enabling the implementation of complex functions in an accessible way for AI developers. Other themes include AI development, interacting with eye models, controlling model generation through prompts, giving a Bot its own identity, language modeling, optimization and management of prompts, customization of prompts, output delivery options, Json format and output parsers, language models and their role in AI, AI applications, switching between open source models, the concept of a chain and its standardized interface, integrations with other tools, complex applications, calls to an LLM (Language Model), high-level chains like retrieval Q&A chains, querying data from a vector database, sending received data to the LLM as additional information, information memory, storage of states between calls, continuity and consistency in processes, memory usage in a chatbot, follow-up 